<a href="https://colab.research.google.com/github/Montse1708/Practica3_Metricas/blob/main/TI_1_Practica3_JessicaMontserratMoralesEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Jessica Montserrat Morales Enrique  
**e-mail:** jessica.morales5556@alumnos.udg.mx

## MODULES

In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [ ]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = #code here
    norm_ab = #code here

    vec_bc = #code here
    norm_bc = #code here

    dot_p = #code here

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = #code here

    # Angle orientation
    cross_p = #code here
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [12]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

###Brownian Motions (BM) utilizando pandas

In [3]:
######################################################################
# Brownian Motion Trajectory
######################################################################

def BM_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  #Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  #Concatenar
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i] + velocity.x, 'y_pos': BM_2d_df.y_pos[i] + velocity.y}])
    #Concatenar
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

###Correlated Random Walks (CRW) utilizando pandas

In [6]:
def CRW_2d(CRW_exponents=0.2, n_steps=1000, speed=5, s_pos=[0,0]):
    # Init velocity vector
    velocity = Vec2d(speed, 0)
    # Guardar las trayectorias en Pandas DataFrame
    CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    # Concatenar
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1 ):
        # Rotación de la trayectoria
        turn_angle = wrapcauchy.rvs(CRW_exponents)
        velocity = velocity.rotated(turn_angle)

        temp_df= pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i] + velocity.x, 'y_pos': CRW_2d_df.y_pos[i] + velocity.y}])
        # Concatenar
        CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
    return CRW_2d_df

###Función auxiliar que sustituye a distance.euclidean

In [17]:
def distance_euclidean(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

###Función alternativa calcula los valores de la curva de path de una trayectoria.

In [15]:
# Define your function to compute path length for given trajectory
## start - Add your code here
def path_length (trajectory):
  n_steps = trajectory.shape[0]
  array_trajectory = [];
  for i in range(1, n_steps):
    array_trajectory.append(distance_euclidean(trajectory.iloc[i-1], trajectory.iloc[i]))
    path_length_df = pd.DataFrame(np.cumsum(array_trajectory), columns=['PL'])
  return path_length_df

## end - Add your code here

In [16]:
# Get Path length calling the function
# Add your code here function(BM_2d_df_3)
PL_BM_3 = path_length(BM_2d_df_3)

# Get Path length calling the function
# Add your code here function(BM_2d_df_6)
PL_BM_6 = path_length(BM_2d_df_6)

# Get Path length calling the function
# Add your code here function(CRW_2d_df_9)
PL_CRW_6 = path_length(BM_2d_df_9)

TypeError: distance_euclidean() missing 2 required positional arguments: 'x2' and 'y2'

In [ ]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here

## end - Add your code here


# Second trace BM2
## start - Add your code here

## end - Add your code here


# Third trace CRW
## start - Add your code here

## end - Add your code here
BM_2d_df_3 = BM_2d(2000,3,[0,0])
BM_2d_df_6 = BM_2d(2000,6,[0,0])
CRW_2d_df_6 = CRW_2d(2000,6,[0,0],0.4)
fig_BM_2d=go.Figure()
PathLenght=calcularDistancia(BM_2d_df_3)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name="BM Walk length vel 3",
    showlegend=True
))
PathLenght=calcularDistancia(BM_2d_df_6)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=6),
    mode='lines',
    name="BM Walk length vel 6",
    showlegend=True
))
PathLenght=calcularDistancia(CRW_2d_df_6)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name='CRW Walk length vel 6',
    showlegend=True
))
fig_BM_2d.show()

fig_path_length.show()

ValueError: CRW_exponents must be in the range (0, 1]

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [ ]:
# Define your function to compute Mean Squared Displacement for given trajectory

## start - Add your code here
# def function (trajectory):


    #return MSD_df

## end - Add your code here

In [ ]:
# Get Mean Squared Displacement calling the function
MSD_BM = # Add your code here function(BM_2d_df_6)

# Get Mean Squared Displacement calling the function
MSD_CRW = # Add your code here function(CRW_2d_df_9)

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here

## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here

## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    ## start - Add your code here

    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))



## start - Add your code here

## end - Add your code here

In [ ]:
# Define your function to compute Turning Angles for given trajectory

## start - Add your code here
# def function (trajectory):


    #return ta_df

## end - Add your code here

In [ ]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = # Add your code here function(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = # Add your code here function(CRW_2d_df_9)

In [ ]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here

## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [ ]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
# def function (trajectory):


    #return sl_df

## end - Add your code here

In [ ]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = # Add your code here function(Levy_2d_df_1)

# Get Step lengths calling the function
sl_Levy_2d_df_7 = # Add your code here function(Levy_2d_df_7)

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here

## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_4.show()